In [7]:
import numpy as np
import pandas as pd

In [8]:
user_nlp = pd.read_csv('data/users_nlp.csv')

In [9]:
user_nlp.head()

,Unnamed: 0,user_id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,...,index,sentiment_negative,sentiment_neutral,sentiment_positive,sent_sum,ratio_neg,ratio_neu,ratio_pos,jaccard,numtops
0,0,64987143.0,Fake Taylor S,TaylorSFaklhh,1,62,1489,0,0,NaN,...,142,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1,72795613.0,Katherine,Katherine9e5b,1,98,411,0,0,http://wowurl.com/13qp,...,173,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,2,53780486.0,Mom3CuteBoys,Mom3CuteBo38991,1,39,285,0,0,NaN,...,76,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
3,3,64982724.0,Nancy,Nancy84pg,1,119,673,0,2,http://xurl.jp/180a,...,141,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,4,69509049.0,Chelsea,Chelsea9j2k,1,189,606,0,0,http://wowurl.com/175,...,166,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
